# Ice-age dynamics: Ice-sheet modelling
----
*Georg Kaufmann,
Geophysics Section,
Institute of Geological Sciences,
Freie Universität Berlin,
Germany*

## Simple plastic ice-sheet model

$$
\frac{\displaystyle dh_{surface}}{\displaystyle dt}
=
\frac{\displaystyle \tau}{\displaystyle \rho g h_{ice}}
$$
with $t$ time, $h_{ice}$ ice-sheet thickness, $h_{surface}$ topographical surface,
$\tau$ shear strength along ice-bedrock interface,
$\rho$ ice density, and
$g$ gravitational acceleration.

In [ ]:
%matplotlib inline
import numpy as np
import scipy
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
from ipywidgets import interactive, fixed, HBox, VBox
import ipywidgets as widgets

# define parameter values
density = 910.  # kg/m3
gravity = 9.81  # m/s2
taur    = 1.e5  # Pa

def plot_profile(t1,t2):
    # define profile kink points (x) and topography (y)
    x = [0.,100.e3,200.e3]
    y = [0.,t1*1000,t2*1000]
    # prepare linear interpolation
    f1 = interp1d(x, y, kind='linear')
    # interpolate to denser grid
    n = 401
    profile = np.linspace(0,200.e3,n,endpoint=True)
    bedrock = f1(profile)
    # define vector fields
    shear = np.full((len(profile)),taur)
    ice   = np.zeros(len(profile))
    topo  = np.zeros(len(profile))
    # fix snout values
    ice[0]  = 0.
    topo[0] = bedrock[0]
    # calculate ice height at second point from parabolic profile
    ice[1]  = np.sqrt(2.*shear[1]/density/gravity*profile[1])
    topo[1] = bedrock[1] + ice[1]
    # loop
    for i in range(2,n):
        topo[i]  = topo[i-1] + shear[i-1] / (density*gravity) * (profile[i]-profile[i-1])/ice[i-1]
        ice[i]   = topo[i] - bedrock[i]
    # plot time series
    plt.figure(figsize=(12.0, 6.0))
    plot1=plt.subplot2grid((3,2), (0,0), colspan=2,rowspan=3)
    #plot2=plt.subplot2grid((3,2), (1,0), colspan=2,rowspan=1)
    #plot3=plt.subplot2grid((3,2), (2,0), colspan=2,rowspan=1)
    plot1.set_xlabel('Profile [km]')
    plot1.set_ylabel('Elevation [km]')
    plot1.set_title('Plastic ice sheet profile')
    plot1.fill_between(profile/1000,-0.1,bedrock/1000,color='brown',linestyle='-',linewidth=1,label='Topo')
    plot1.fill_between(profile/1000,bedrock/1000,topo/1000,color='blue',linestyle='-',linewidth=1,label='Ice')
    plot1.set_xlim([0,200])
    plot1.set_ylim([-0.1,4.0])
    #plot1.axes.get_xaxis().set_visible(False)
    #plot1.spines['bottom'].set_visible(False)
    plot1.legend(loc='upper left')
    
#plot_profile(1000,1500)

# call interactive module
interactive_plot = interactive(plot_profile,
                               t1=widgets.FloatSlider(min=0,max=3,step=0.1,value=0,description='t1 [km]'),
                               t2=widgets.FloatSlider(min=0,max=3,step=0.1,value=0,description='t2 [km]'))
output = interactive_plot.children[-1]
interactive_plot

## Ice-sheet models with mass balance
$$
 \frac{\displaystyle dh_{surface}}{\displaystyle dx} = \frac{\displaystyle A_s}{\displaystyle \rho g}
 \frac{\displaystyle \left[ a(L-R) + b(R-x) \right]^{1 \over m}}{\displaystyle h_{ice}^{1+{1 \over m}}}
$$

In [ ]:
%matplotlib inline
import numpy as np
import scipy
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
from ipywidgets import interactive, fixed, HBox, VBox
import ipywidgets as widgets

# define parameter values
density = 910.  # kg/m3
gravity = 9.81  # m/s2
taur    = 1.e5  # Pa
am      = 40.
m       = 2.5
a       = am**m

def plot_profile(t1,t2,ela,abl,acc):
    # define profile kink points (x) and topography (y)
    x = [0.,100.e3,200.e3]
    y = [0.,t1*1000,t2*1000]
    # prepare linear interpolation
    f1 = interp1d(x, y, kind='linear')
    # interpolate to denser grid
    n = 401
    profile = np.linspace(0,200.e3,n,endpoint=True)
    bedrock = f1(profile)
    # define vector fields
    shear = np.full((len(profile)),taur)
    ice   = np.zeros(len(profile))
    topo  = np.zeros(len(profile))
    # fix snout values
    ice[0]  = 0.
    topo[0] = bedrock[0]
    # calculate ice height at second point from parabolic profile
    ice[1]  = np.sqrt(2.*shear[1]*profile[1]/density/gravity)
    topo[1] = bedrock[1] + ice[1]
    # loop
    for i in range(2,n):
        dx = profile[i] - profile[i-1]
        #if (profile[i] <= ela*1000):
        #    dheight     = a * (abl*float(i)*dx)**(1./m) / (density*gravity)/ice[i-1]**(1.+1./m)*dx
        #else:
        #    dheight     = a * (acc*(profile[n-1]-float(i)*dx))**(1./m) / (density*gravity)/ice[i-1]**(1.+1./m)*dx
            
        if (profile[i] <= ela*1000):
            dheight     = (a/(density*gravity)/ice[i-1]**(1.+1./m)*dx 
                        * (-abl*(ela*1000-float(i)*dx) + acc*(profile[n-1]-ela*1000))**(1./m))
        else:
            dheight     = (a/(density*gravity)/ice[i-1]**(1.+1./m)*dx 
                        * (acc*(profile[n-1]-ela*1000))**(1./m))
            
        topo[i]  = topo[i-1] + dheight
        ice[i]   = topo[i] - bedrock[i]
    # plot time series
    plt.figure(figsize=(12.0, 6.0))
    plot1=plt.subplot2grid((3,2), (0,0), colspan=2,rowspan=3)
    #plot2=plt.subplot2grid((3,2), (1,0), colspan=2,rowspan=1)
    #plot3=plt.subplot2grid((3,2), (2,0), colspan=2,rowspan=1)
    plot1.set_xlabel('Profile [km]')
    plot1.set_ylabel('Elevation [km]')
    plot1.set_title('Plastic ice sheet profile')
    plot1.fill_between(profile/1000,-0.1,bedrock/1000,color='brown',linestyle='-',linewidth=1,label='Topo')
    plot1.fill_between(profile/1000,bedrock/1000,topo/1000,color='blue',linestyle='-',linewidth=1,label='Ice')
    plot1.set_xlim([0,200])
    plot1.set_ylim([-0.1,4.0])
    #plot1.axes.get_xaxis().set_visible(False)
    #plot1.spines['bottom'].set_visible(False)
    plot1.legend(loc='upper left')
    
#plot_profile(1000,1500)

# call interactive module
interactive_plot = interactive(plot_profile,
                               t1=widgets.FloatSlider(min=0,max=3,step=0.1,value=0,description='t1 [km]'),
                               t2=widgets.FloatSlider(min=0,max=3,step=0.1,value=0,description='t2 [km]'),
                               ela=widgets.IntSlider(min=0,max=200,step=10,value=0,description='ELA [km]'),
                               abl=widgets.FloatSlider(min=0,max=2,step=0.1,value=1,description='Ablation-zone [m/yr]'),
                               acc=widgets.FloatSlider(min=0,max=3,step=0.1,value=2,description='Accumulation-zone [m/yr]'))
output = interactive_plot.children[-1]
interactive_plot